# Check missing cmip5 dataset

Reported issue:

```
dataset = "projections-cmip5-daily-single-levels"
request = {
    "experiment": "rcp_8_5",
    "variable": ["2m_temperature"],
    "model": "cesm1_cam5",
    "ensemble_member": "r1i1p1",
    "period": ["20760101-21001231"]
}
```


## Check download

In [1]:
import requests
from requests import ConnectionError

In [2]:
def check_cmip5(host=None):
    host = host or "data.mips.climate.copernicus.eu"
    ds_file = "tas_day_CESM1-CAM5_rcp85_r1i1p1_20760101-21001231.nc"
    ds_path = "output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/day/atmos/day/r1i1p1/tas/latest"
    url = f"http://{host}/thredds/fileServer/esg_c3s-cmip5/{ds_path}/{ds_file}"
    try:
        resp = requests.head(url)
    except ConnectionError as err:
        print(err)
    else:
        print(resp.ok)
        print(resp.headers)
        bytes = int(resp.headers.get('Content-Length'))
        print(f"{bytes} bytes, {int(bytes/1024/1024)} MB")

In [3]:
# CDS LB
check_cmip5()

True
{'Server': 'nginx/1.20.1', 'Date': 'Tue, 21 Jan 2025 14:56:36 GMT', 'Content-Type': 'application/octet-stream', 'Content-Length': '2018545492', 'Last-Modified': 'Tue, 10 Jul 2018 02:07:06 GMT', 'Connection': 'keep-alive', 'ETag': '"5b4414ca-78508f54"', 'Accept-Ranges': 'bytes'}
2018545492 bytes, 1925 MB


In [4]:
# DKRZ
check_cmip5("136.172.60.74")

True
{'Server': 'nginx/1.20.1', 'Date': 'Tue, 21 Jan 2025 14:56:36 GMT', 'Content-Type': 'application/octet-stream', 'Content-Length': '2018545492', 'Last-Modified': 'Tue, 10 Jul 2018 02:07:06 GMT', 'Connection': 'keep-alive', 'ETag': '"5b4414ca-78508f54"', 'Accept-Ranges': 'bytes'}
2018545492 bytes, 1925 MB


In [5]:
# IPSL
check_cmip5("134.157.240.31")

True
{'Server': 'nginx/1.14.1', 'Date': 'Tue, 21 Jan 2025 14:56:36 GMT', 'Content-Type': 'application/octet-stream', 'Content-Length': '2018545492', 'Last-Modified': 'Sat, 20 Mar 2021 08:52:48 GMT', 'Connection': 'keep-alive', 'ETag': '"6055b7e0-78508f54"', 'Accept-Ranges': 'bytes'}
2018545492 bytes, 1925 MB


In [6]:
# old CDS LB

check_cmip5("data.mips.copernicus-climate.eu")

HTTPConnectionPool(host='data.mips.copernicus-climate.eu', port=80): Max retries exceeded with url: /thredds/fileServer/esg_c3s-cmip5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/day/atmos/day/r1i1p1/tas/latest/tas_day_CESM1-CAM5_rcp85_r1i1p1_20760101-21001231.nc (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x110d4d490>: Failed to resolve 'data.mips.copernicus-climate.eu' ([Errno 8] nodename nor servname provided, or not known)"))


## Check subsetting

In [7]:
from rooki import operators as ops

In [8]:
wf = ops.Subset(
        ops.Input(
            'tas', ['c3s-cmip5.output1.NSF-DOE-NCAR.CESM1-CAM5.rcp85.day.atmos.day.r1i1p1.tas.latest']
        ),
        time="2077-01/2077-01",
)
resp = wf.orchestrate()
resp.ok

True

In [9]:
ds = resp.datasets()[0]
ds

<xarray.Dataset> Size: 7MB
Dimensions:    (time: 31, bnds: 2, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 248B 2077-01-01 12:00:00 ... 2077-01-31 12:00:00
  * lat        (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
    height     float64 8B ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 496B ...
    lat_bnds   (lat, bnds) float64 3kB ...
    lon_bnds   (lon, bnds) float64 5kB ...
    tas        (time, lat, lon) float32 7MB ...
Attributes: (12/36)
    institution:                  NSF/DOE NCAR (National Center for Atmospher...
    institute_id:                 NSF-DOE-NCAR
    experiment_id:                rcp85
    source:                       CESM1-CAM5
    model_id:                     CESM1-CAM5
    forcing:                      Sl GHG SS Ds SA BC MD OC Oz AA
    ...                           ...
    table_id:                     Table day (12 January 2012) 7757d80c56ae0b9...
    title:                        CESM1-CAM5 model output prepared for CMIP5 ...
    parent_experiment:            historical
    modeling_realm:               atmos
    realization:                  1
    cmor_version:                 2.8.1